In [1]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle
from Bio import AlignIO, SeqIO
from ete3 import Tree, TreeNode
from gctree import CollapsedTree

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from ete3 import Tree, faces, TreeStyle, NodeStyle, TextFace, SequenceFace, COLOR_SCHEMES, CircleFace
from GCTree_preparation import *
import warnings
import math
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNHD01/storage/all_BSimons_datasets"
outdir = "/media/hieunguyen/GSHD_HN01/outdir/sc_bulk_BCR_data_analysis_v0.1"

PROJECT = "241031_BSimons"
path_to_main_output = f"{outdir}/tree_analysis/{PROJECT}"
path_to_01_output = os.path.join(path_to_main_output, "01_output")
os.system(f"mkdir -p {path_to_01_output}")

output_type = "all"

path_to_trees = os.path.join(path_to_storage, PROJECT, "GCtrees/v0.2", output_type)

all_tree_folder = [item for item in pathlib.Path(path_to_trees).glob("*/*") if 
                   os.path.isfile(f"{str(item)}/02_dnapars/gctree.out.inference.1.nk") == True]

all_nk_files = [item for item in pathlib.Path(path_to_trees).glob("*/*/*gctree.out.inference.1.nk")]  
print(f"Number of trees: {len(all_tree_folder)}")   

##### Re run the summary analysis of all trees and rendering tree figures
# rerun = True
rerun = False

path_to_04_output = os.path.join(outdir, "VDJ_output", "04_output")
thres = 0.85

clonedf = pd.read_csv(os.path.join(path_to_04_output, "full_clonedf_with_mutation_rate.csv"), index_col= [0])
clonedf = clonedf[clonedf['num_mutation'] != "region_not_covered-skip"]
clonedf = clonedf[clonedf['dataset.name'] == PROJECT]

mid_metadata = pd.read_excel("/media/hieunguyen/HNSD01/src/sc_bulk_BCR_data_analysis/preprocessing/241031_BSimons/241031_sample_sheet.xlsx")
mid_metadata.columns = ["MID", "mouse", "organ", "YFP", "population"]

/home/hieunguyen/miniconda3/envs/ete3_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of trees: 446


In [2]:
if (os.path.isfile(f"{path_to_01_output}/tree_summarydf.csv") == False) or (rerun == True):
    saveTreeobj = dict()
    # loop through all trees
    maindf = pd.DataFrame()
    # for treedir in tqdm(all_tree_folder[0:1]):
    for treedir in tqdm(all_tree_folder):
        cloneid = treedir.name
        treedir = str(treedir)

        mouseID = treedir.split("/")[-2].split("_")[0]
        V_gene = cloneid.split("_")[0]
        J_gene = cloneid.split("_")[1]
        CDR3_len = cloneid.split("_")[2]

        nk_path = f"{treedir}/02_dnapars/gctree.out.inference.1.nk"
        ab_dict_path = os.path.join(treedir, "01_deduplicate", f"{cloneid}.abundance.csv")
        path_to_orig_fasta= f"{treedir}/01_deduplicate/{cloneid}.fasta"
        input_idmaps = f"{treedir}/01_deduplicate/{cloneid}.id_map_seq.csv"

        treeobj = GCtree(
            nk_path = nk_path,
            ab_dict_path = ab_dict_path,
            origin_fasta = path_to_orig_fasta,
            idmap_seq = input_idmaps
        )

        saveTreeobj[f"{mouseID}_{cloneid}"] = treeobj
        seqdf_orig = treeobj.seqdf
        seqs = treeobj.seqs
        num_nodes = len(treeobj.nodes)
        num_leaves = len(treeobj.leaves)
        num_internal_nodes = len(treeobj.internal_nodes)
        num_passthrough_nodes = len(treeobj.passthrough_nodes)
        num_split_nodes = len(treeobj.split_nodes)
        num_observed_nodes = len(treeobj.observed_nodes)
        num_inferred_nodes = len(treeobj.inferred_nodes)
        count_single_node = treeobj.count_single_node
        count_mix_node = treeobj.count_mix_node
        all_MIDs = seqdf_orig["MID"].unique()
        all_groups = [mid_metadata[mid_metadata["MID"] == item]["population"].unique()[0] for item in all_MIDs]
        tmpdf = pd.DataFrame(
            {
                "cloneid": f"{mouseID}_{cloneid}",
                "mouseID": mouseID,
                "V_gene": V_gene,
                "J_gene": J_gene,
                "CDR3_len": CDR3_len,
                "num_nodes": num_nodes,
                "num_leaves": num_leaves,
                "num_internal_nodes": num_internal_nodes,
                "num_passthrough_nodes": num_passthrough_nodes,
                "num_split_nodes": num_split_nodes,
                "num_observed_nodes": num_observed_nodes,
                "num_inferred_nodes": num_inferred_nodes,
                "num_MID": len(all_MIDs),
                "available_population": ",".join(all_groups),
                "num_seq_fasta": len(seqs),
                "num_single_node": count_single_node,
                "num_mix_node": count_mix_node
            },
            index=[0],
        )
        maindf = pd.concat([maindf, tmpdf], axis = 0)
    maindf = maindf.sort_values(by = "num_MID", ascending = False)
    maindf["check_QC_tree"] = maindf["cloneid"].apply(lambda x: "CDR3 region not covered" if "region_not_cover" in saveTreeobj[x].seqdf.ID.values[0] else "pass")
    maindf.to_csv(f"{path_to_01_output}/tree_summarydf.csv", index = False)
    with open(f"{path_to_01_output}/saveTreeobj.pkl", "wb") as f:
        pickle.dump(saveTreeobj, f)
else:
    maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.csv")
    # Reload the dictionary from the pickle file
    with open(f"{path_to_01_output}/saveTreeobj.pkl", "rb") as f:
        saveTreeobj = pickle.load(f)

mid_metadata["color"] = ["#F8766D",
                          "#DE8C00",
                          "#B79F00",
                          "#7CAE00",
                          "#00BA38",
                          "#00C08B",
                          "#00BFC4",
                          "#00B4F0"]
colordf = mid_metadata[["MID", "color"]]
colordf.columns = ["MID", "hex color"]
colordf.to_csv(f"{PROJECT}_color.csv", header = True, index=False)

color_path = f"{PROJECT}_color.csv"
mid_metadata["MID"] = mid_metadata["MID"].apply(lambda x: f"MID{x}")
def get_tree_sum_abundance(x):
    mouseid = x.split("_")[0]
    path_to_save_tree_svg = os.path.join(path_to_01_output, mouseid)
    os.system(f"mkdir -p {path_to_save_tree_svg}")

    treeobj = saveTreeobj[x] 

    idmapdf = treeobj.idmapseqdf.copy()
    seqdf = treeobj.seqdf.copy()
    seqdf["population"] = seqdf["MID"].apply(lambda x: mid_metadata[mid_metadata["MID"] == x]["population"].values[0])
    seqdf = seqdf.merge(idmapdf, right_on = "seq", left_on = "seq")
    abd = seqdf.abundance.sum()
    return abd

tqdm.pandas()
# Reload the dictionary from the pickle file
with open(f"{path_to_01_output}/saveTreeobj.pkl", "rb") as f:
    saveTreeobj = pickle.load(f)
if os.path.isfile(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv") == False:
    maindf["clone_sum_abundance"] = maindf["cloneid"].progress_apply(lambda x: get_tree_sum_abundance(x))
    maindf["pct_sum_abundance"] = maindf[["clone_sum_abundance", "mouseID"]].progress_apply(lambda x: x[0]/maindf[maindf["mouseID"] == x[1]]["clone_sum_abundance"].sum(), axis = 1)
    maindf = maindf.sort_values(by=  "pct_sum_abundance", ascending = False)
    maindf.to_csv(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv", index=False)
else:
    maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv")

  0%|          | 1/446 [00:00<00:01, 227.96it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
if (os.path.isfile(f"{path_to_01_output}/finished_tree_rendering.csv") == False) or (rerun == True):
    for cloneid in tqdm(saveTreeobj.keys()):
        mouseid = cloneid.split("_")[0]
        path_to_save_tree_svg = os.path.join(path_to_01_output, mouseid)
        os.system(f"mkdir -p {path_to_save_tree_svg}")

        treeobj = saveTreeobj[cloneid] 
        avai_mids = treeobj.seqdf["MID"].unique()
        mid_color_pal = pd.read_csv(color_path, index_col = [0]).to_dict()["hex color"]

        ts = treeobj.generate_tree_style(color_path = color_path)
        # treeobj.tree.render("%%inline", tree_style=ts) 

        for input_mid in avai_mids:
            if input_mid == "GL":
                input_mid_col = "gray"
            else:
                input_mid_col = mid_color_pal[input_mid]
            ts.legend.add_face(CircleFace(10, input_mid_col), column = 0)
            ts.legend.add_face(TextFace(input_mid), column = 0)

        _ = treeobj.tree.render(f"{path_to_save_tree_svg}/{cloneid}.svg", tree_style=ts) 
    pd.DataFrame(data = ["finished_tree_rendering"]).to_csv(f"{path_to_01_output}/finished_tree_rendering.csv", index = False, header = False)